##### USING TF-IDF AND COSINE SIMILARITY WITH PARALLEL PROCESSING

History on runtime based on parameter and CPU cores
- chunk_size = 1000
- *cpu_core = -2 OR cpu_core = -3 (no difference in runtime both took ~15min to complete)

*n_jobs=-3, it means that joblib will use all but three cores of the CPU for running the tasks. This is because joblib uses negative values of n_jobs to specify the number of cores to keep idle. So, -3 means keep 3 cores idle and use the rest.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed
import pandas as pd
import re
from datetime import datetime
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import multiprocessing
import gc

print('Just checking how many rows are there in the source excelfile...')
raw_df_vendor_customer = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/Vendor Customer.xlsx')
raw_df_interested_party = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/Interested Parties.xlsx')
print('Number of rows in raw_df_vendor_customer', len(raw_df_vendor_customer))
print('Number of rows in raw_df_interested_party', len(raw_df_interested_party))


print('ANALYSES BEGINS! :)')
# loading and reading into dataframe
df_vendor_customer = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/Vendor Customer.xlsx', nrows=100000)
df_interested_party = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/Interested Parties.xlsx', nrows=100000)
print('Complete: Data loaded into DF', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print('Number of rows in df_vendor_customer', len(df_vendor_customer))
print('Number of rows in df_interested_party', len(df_interested_party))


print('Start: Data Massaging', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# set the index as a column, to be used as a mapping field to join df_vendor_customer
df_interested_party = df_interested_party.reset_index().rename(columns={'index': 'index_ID'})

# to replace NULL/NaN values with empty strings
df_vendor_customer['Name'] = df_vendor_customer['Name'].fillna('')
df_interested_party['Interested Party List'] =df_interested_party['Interested Party List'].fillna('')


# define a regular expression that matches all non-alphanumeric and non-space characters and remove them
pattern = re.compile(r'[^\w\s]+')

df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name'].apply(lambda x: re.sub(pattern, '', x))
df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List'].apply(lambda x: re.sub(pattern, '', x))


# update strings to all uppercase()
df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name_Cleaned'].str.upper()
df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List_Cleaned'].str.upper()


# define the list of common words to remove, to remove noise (similar to stopwords concept)
# create a regular expression pattern that includes word boundaries (\b) before and after each word in the list of words to remove. This ensures that the str.replace method only removes the word when it appears as a standalone word, and not as a substring of other words.
words_to_remove = ['PTE', 'LTD', 'LLC', 'CO', 'SDN', 'BHD', 'PTY LIMITED', 'PTY', 'LIMITED', 'PVT', 'PRIVATE', 'INC', 'LLP', 'COMPANY']
pattern = r'\b(' + '|'.join(words_to_remove) + r')\b'


for word in words_to_remove:
    df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name_Cleaned'].str.replace(pattern, '', regex=True)
    df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List_Cleaned'].str.replace(pattern, '', regex=True)


# update strings to remove leading and trailing whitespaces
df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name_Cleaned'].str.strip()
df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List_Cleaned'].str.strip()

# to drop duplicated rows
df_vendor_customer = df_vendor_customer.drop_duplicates()
df_interested_party = df_interested_party.drop_duplicates()

print('Complete: Data Massaging', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print('Number of rows in df_vendor_customer after data massaging', len(df_vendor_customer))
print('Number of rows in df_interested_party after data massaging', len(df_interested_party))


print('Start: Vectorisation', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# concatenate the two columns separately to have same number of rows in each list (difference in the number of rows between the two datasets). This will cause memory error.
# use numpy arrays instead of Pandas dataframes to reduce memory usage
concatenated_vendor_customer = np.concatenate([df_vendor_customer['Name_Cleaned'].values, np.array([''] * len(df_interested_party))])
concatenated_interested_party = np.concatenate([df_interested_party['Interested Party List_Cleaned'].values, np.array([''] * len(df_vendor_customer))])
# concatenated_vendor_customer = pd.concat([df_vendor_customer['Name_Cleaned'], pd.Series([''] * len(df_interested_party))])
# concatenated_interested_party = pd.concat([df_interested_party['Interested Party List_Cleaned'], pd.Series([''] * len(df_vendor_customer))])


# vectorize the 'Name' and 'Interested Party List' columns using TF-IDF
tfidf_vectorizer = TfidfVectorizer()

tfidf_vectorizer.fit(concatenated_vendor_customer + " " + concatenated_interested_party)

tfidf_matrix_a = tfidf_vectorizer.transform(df_vendor_customer['Name_Cleaned']).toarray()
tfidf_matrix_b = tfidf_vectorizer.transform(df_interested_party['Interested Party List_Cleaned']).toarray()
print('Complete: Vectorisation', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


# Define the chunk size and number of jobs for parallel processing
chunk_size = 1000
num_jobs = -1 # number of CPU cores to use for parallel processing
print('Number of CPU cores available:', multiprocessing.cpu_count(), 'Number of CPU cores to use:', num_jobs)


# Define a function to compute similarities for a chunk of vendor names
def compute_similarities(chunk):
    
    # Compute the similarity matrix for the chunk
    similarity_matrix = cosine_similarity(chunk, tfidf_matrix_b)

    # Find the index and value of the interested party name with the highest similarity for each vendor name in the chunk
    max_similarities = np.argmax(similarity_matrix, axis=1)
    max_similarity_scores = np.max(similarity_matrix, axis=1)
    
    # Return the corresponding interested party names, similarity scores and interested party source for the chunk
    return df_interested_party['Interested Party List_Cleaned'].iloc[max_similarities].values, max_similarity_scores, df_interested_party['Interested Party Source'].iloc[max_similarities].values, df_interested_party['Interested Party List'].iloc[max_similarities].values


print('Start: Parallel Processing', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
results = Parallel(n_jobs=num_jobs)(delayed(compute_similarities)(chunk) for chunk in np.array_split(tfidf_matrix_a, len(df_vendor_customer)//chunk_size+1)) # Split the vendor names into chunks and compute similarities in parallel
print('End: Parallel Processing', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


# Combine the results for all chunks into a single dataframe
corresponding_names = np.concatenate([result[0] for result in results])
similarity_scores = np.concatenate([result[1] for result in results])
corresponding_interested_party_source = np.concatenate([result[2] for result in results])
corresponding_names_original = np.concatenate([result[3] for result in results])


# Add the corresponding interested party names and similarity scores to the vendor customer dataframe
df_vendor_customer['Corresponding Interested Party Name'] = corresponding_names
df_vendor_customer['Cosine Similarity Score'] = similarity_scores 
df_vendor_customer['Corresponding Interested Party Source'] = corresponding_interested_party_source
df_vendor_customer['Corresponding Interested Party Name (Original)'] = corresponding_names_original


# Write to excel file
df_vendor_customer.to_excel('results_100k.xlsx')
print('Complete: Results to Excel', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

# To free up memory
gc.collect()

# 4000 rows took 20 seconds with 3 CPU cores
# 100,000 rows took 15 minutes with 3 CPU cores - cannot go beyond 100k rows due to insufficient RAM to create all vectors.
# 86,197 rows took 7 minutes with 3 CPU cores


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed
import pandas as pd
import re
from datetime import datetime
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import multiprocessing
import gc

print('Just checking how many rows are there in the source excelfile...')
raw_df_vendor_customer = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/Vendor Customer.xlsx')
raw_df_interested_party = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/Interested Parties.xlsx')
print('Number of rows in raw_df_vendor_customer', len(raw_df_vendor_customer))
print('Number of rows in raw_df_interested_party', len(raw_df_interested_party))


print('ANALYSES BEGINS! :)')
# loading and reading into dataframe
df_vendor_customer = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/Vendor Customer.xlsx', skiprows= range(1, 100001))
df_interested_party = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/Interested Parties.xlsx', nrows=86197)
print('Complete: Data loaded into DF', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print('Number of rows in df_vendor_customer', len(df_vendor_customer))
print('Number of rows in df_interested_party', len(df_interested_party))


print('Start: Data Massaging', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# set the index as a column, to be used as a mapping field to join df_vendor_customer
df_interested_party = df_interested_party.reset_index().rename(columns={'index': 'index_ID'})

# to replace NULL/NaN values with empty strings
df_vendor_customer['Name'] = df_vendor_customer['Name'].fillna('')
df_interested_party['Interested Party List'] =df_interested_party['Interested Party List'].fillna('')


# define a regular expression that matches all non-alphanumeric and non-space characters and remove them
pattern = re.compile(r'[^\w\s]+')

df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name'].apply(lambda x: re.sub(pattern, '', x))
df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List'].apply(lambda x: re.sub(pattern, '', x))


# update strings to all uppercase()
df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name_Cleaned'].str.upper()
df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List_Cleaned'].str.upper()


# define the list of common words to remove, to remove noise (similar to stopwords concept)
# create a regular expression pattern that includes word boundaries (\b) before and after each word in the list of words to remove. This ensures that the str.replace method only removes the word when it appears as a standalone word, and not as a substring of other words.
words_to_remove = ['PTE', 'LTD', 'LLC', 'CO', 'SDN', 'BHD', 'PTY LIMITED', 'PTY', 'LIMITED', 'PVT', 'PRIVATE', 'INC', 'LLP', 'COMPANY']
pattern = r'\b(' + '|'.join(words_to_remove) + r')\b'


for word in words_to_remove:
    df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name_Cleaned'].str.replace(pattern, '', regex=True)
    df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List_Cleaned'].str.replace(pattern, '', regex=True)


# update strings to remove leading and trailing whitespaces
df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name_Cleaned'].str.strip()
df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List_Cleaned'].str.strip()

# to drop duplicated rows
df_vendor_customer = df_vendor_customer.drop_duplicates()
df_interested_party = df_interested_party.drop_duplicates()

print('Complete: Data Massaging', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print('Number of rows in df_vendor_customer after data massaging', len(df_vendor_customer))
print('Number of rows in df_interested_party after data massaging', len(df_interested_party))


print('Start: Vectorisation', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# concatenate the two columns separately to have same number of rows in each list (difference in the number of rows between the two datasets). This will cause memory error.
# use numpy arrays instead of Pandas dataframes to reduce memory usage
concatenated_vendor_customer = np.concatenate([df_vendor_customer['Name_Cleaned'].values, np.array([''] * len(df_interested_party))])
concatenated_interested_party = np.concatenate([df_interested_party['Interested Party List_Cleaned'].values, np.array([''] * len(df_vendor_customer))])
# concatenated_vendor_customer = pd.concat([df_vendor_customer['Name_Cleaned'], pd.Series([''] * len(df_interested_party))])
# concatenated_interested_party = pd.concat([df_interested_party['Interested Party List_Cleaned'], pd.Series([''] * len(df_vendor_customer))])


# vectorize the 'Name' and 'Interested Party List' columns using TF-IDF
tfidf_vectorizer = TfidfVectorizer()

tfidf_vectorizer.fit(concatenated_vendor_customer + " " + concatenated_interested_party)

tfidf_matrix_a = tfidf_vectorizer.transform(df_vendor_customer['Name_Cleaned']).toarray()
tfidf_matrix_b = tfidf_vectorizer.transform(df_interested_party['Interested Party List_Cleaned']).toarray()
print('Complete: Vectorisation', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


# Define the chunk size and number of jobs for parallel processing
chunk_size = 1000
num_jobs = -3 # number of CPU cores to use for parallel processing
print('Number of CPU cores available:', multiprocessing.cpu_count(), '--> Number of CPU cores to use:', num_jobs)


# Define a function to compute similarities for a chunk of vendor names
def compute_similarities(chunk):
    
    # Compute the similarity matrix for the chunk
    similarity_matrix = cosine_similarity(chunk, tfidf_matrix_b)

    # Find the index and value of the interested party name with the highest similarity for each vendor name in the chunk
    max_similarities = np.argmax(similarity_matrix, axis=1)
    max_similarity_scores = np.max(similarity_matrix, axis=1)
    
    # Return the corresponding interested party names, similarity scores and interested party source for the chunk
    return df_interested_party['Interested Party List_Cleaned'].iloc[max_similarities].values, max_similarity_scores, df_interested_party['Interested Party Source'].iloc[max_similarities].values, df_interested_party['Interested Party List'].iloc[max_similarities].values


print('Start: Parallel Processing', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
results = Parallel(n_jobs=num_jobs)(delayed(compute_similarities)(chunk) for chunk in np.array_split(tfidf_matrix_a, len(df_vendor_customer)//chunk_size+1)) # Split the vendor names into chunks and compute similarities in parallel
print('End: Parallel Processing', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


# Combine the results for all chunks into a single dataframe
corresponding_names = np.concatenate([result[0] for result in results])
similarity_scores = np.concatenate([result[1] for result in results])
corresponding_interested_party_source = np.concatenate([result[2] for result in results])
corresponding_names_original = np.concatenate([result[3] for result in results])


# Add the corresponding interested party names and similarity scores to the vendor customer dataframe
df_vendor_customer['Corresponding Interested Party Name'] = corresponding_names
df_vendor_customer['Cosine Similarity Score'] = similarity_scores 
df_vendor_customer['Corresponding Interested Party Source'] = corresponding_interested_party_source
df_vendor_customer['Corresponding Interested Party Name (Original)'] = corresponding_names_original


# Write to excel file
df_vendor_customer.to_excel('results_86k.xlsx')
print('Complete: Results to Excel', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

# To free up memory
gc.collect()

# 4000 rows took 20 seconds with 3 CPU cores
# 100,000 rows took 15 minutes with 3 CPU cores - cannot go beyond 100k rows due to insufficient RAM to create all vectors.
# 86,197 rows took 7 minutes with 3 CPU cores


In [ ]:

# combined multiple excelfile into single excelfile
df1 = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/results_100k.xlsx')
df2 = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/results_86k.xlsx')

combined_df = pd.concat([df1, df2])

combined_df.to_excel('combined_results.xlsx', index=False)

# 177k rows in total - 1min

##### USING LEVENSHTEIN DISTANCE WITH PARALLEL PROCESSING

- 4000 rows with -3 CPU cores, 1000 chunk size took 3min
- 186k rows with -3 CPU cores, 1000 chunk size took 182min (3hours)

In [ ]:
# using fuzzywuzzy based on Levenshtein Distance

import pandas as pd
import multiprocessing 
from fuzzywuzzy import fuzz
import re
import numpy as np
from datetime import datetime
from joblib import Parallel, delayed


# Load the dataframes
print('Just checking how many rows are there in the source excelfile...')
raw_df_vendor_customer = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/Vendor Customer.xlsx')
raw_df_interested_party = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/Interested Parties.xlsx')
print('Number of rows in raw_df_vendor_customer', len(raw_df_vendor_customer))
print('Number of rows in raw_df_interested_party', len(raw_df_interested_party))


print('ANALYSES BEGINS! :)')
# loading and reading into dataframe
df_vendor_customer = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/Vendor Customer.xlsx')
df_interested_party = pd.read_excel('C:/Desktop/Repo/Others/interparty related matching/Interested Parties.xlsx')
print('Complete: Data loaded into DF', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print('Number of rows in df_vendor_customer', len(df_vendor_customer))
print('Number of rows in df_interested_party', len(df_interested_party))


print('Start: Data Massaging', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# set the index as a column, to be used as a mapping field to join df_vendor_customer
df_interested_party = df_interested_party.reset_index().rename(columns={'index': 'index_ID'})

# to replace NULL/NaN values with empty strings
df_vendor_customer['Name'] = df_vendor_customer['Name'].fillna('')
df_interested_party['Interested Party List'] =df_interested_party['Interested Party List'].fillna('')


# define a regular expression that matches all non-alphanumeric and non-space characters and remove them
pattern = re.compile(r'[^\w\s]+')

df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name'].apply(lambda x: re.sub(pattern, '', x))
df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List'].apply(lambda x: re.sub(pattern, '', x))


# update strings to all uppercase()
df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name_Cleaned'].str.upper()
df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List_Cleaned'].str.upper()


# define the list of common words to remove, to remove noise (similar to stopwords concept)
# create a regular expression pattern that includes word boundaries (\b) before and after each word in the list of words to remove. This ensures that the str.replace method only removes the word when it appears as a standalone word, and not as a substring of other words.
words_to_remove = ['PTE', 'LTD', 'LLC', 'CO', 'SDN', 'BHD', 'PTY LIMITED', 'PTY', 'LIMITED', 'PVT', 'PRIVATE', 'INC', 'LLP', 'COMPANY']
pattern = r'\b(' + '|'.join(words_to_remove) + r')\b'


for word in words_to_remove:
    df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name_Cleaned'].str.replace(pattern, '', regex=True)
    df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List_Cleaned'].str.replace(pattern, '', regex=True)


# update strings to remove leading and trailing whitespaces
df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name_Cleaned'].str.strip()
df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List_Cleaned'].str.strip()

# to drop duplicated rows
df_vendor_customer = df_vendor_customer.drop_duplicates()
df_interested_party = df_interested_party.drop_duplicates()

print('Complete: Data Massaging', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


# Define the chunk size and number of jobs for parallel processing
chunk_size = 1000
num_jobs = -3 # number of CPU cores to use for parallel processing
print('Number of CPU cores available:', multiprocessing.cpu_count(), '--> Number of CPU cores to use:', num_jobs)

# Define a function to compute similarities for a chunk of vendor names using fuzzywuzzy
def compute_similarities_fuzzy(chunk):

    # Compute the similarity scores between each vendor name and each interested party name in the chunk
    similarities = [[fuzz.token_sort_ratio(vendor, party) for party in df_interested_party['Interested Party List_Cleaned']] for vendor in chunk]

    # Find the index and value of the interested party name with the highest similarity score for each vendor name in the chunk
    max_similarities = np.argmax(similarities, axis=1)
    max_similarity_scores = np.max(similarities, axis=1)
    
    # Return the corresponding interested party names, similarity scores and interested party source for the chunk
    return df_interested_party['Interested Party List_Cleaned'].iloc[max_similarities].values, max_similarity_scores, df_interested_party['Interested Party Source'].iloc[max_similarities].values, df_interested_party['Interested Party List'].iloc[max_similarities].values

print('Start: Parallel Processing', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
results = Parallel(n_jobs=num_jobs)(delayed(compute_similarities_fuzzy)(chunk) for chunk in np.array_split(df_vendor_customer['Name_Cleaned'].values, len(df_vendor_customer)//chunk_size+1)) # Split the vendor names into chunks and compute similarities in parallel using fuzzywuzzy
print('End: Parallel Processing', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


# Combine the results for all chunks into a single dataframe
corresponding_names = np.concatenate([result[0] for result in results])
similarity_scores = np.concatenate([result[1] for result in results])
corresponding_interested_party_source = np.concatenate([result[2] for result in results])
corresponding_names_original = np.concatenate([result[3] for result in results])


# Add the corresponding interested party names and similarity scores to the vendor customer dataframe
df_vendor_customer['Corresponding Interested Party Name'] = corresponding_names
df_vendor_customer['Fuzzy Score'] = similarity_scores 
df_vendor_customer['Corresponding Interested Party Source'] = corresponding_interested_party_source
df_vendor_customer['Corresponding Interested Party Name (Original)'] = corresponding_names_original


# Write to excel file
df_vendor_customer.to_excel('result_fuzzywuzzy_PP.xlsx', index=False)
print('Complete: Results to Excel', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

# 4000 rows with -3 CPU cores, 1000 chunk size took 3min
# 186k rows with -3 CPU cores, 1000 chunk size took 182min (3hours)
